In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests

In [2]:
# URL of page to be scraped
url = 'https://www.qsrmagazine.com/reports/qsr50-2013-top-50-chart'

In [3]:
# Retrieve page with the requests module
response = requests.get(url)

In [4]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(response.text, 'html.parser')

In [5]:
# Examine the results, then determine element that contains sought info
print(soup.prettify())

               Fast Food
               </a>
              </a>
             </h2>
             <div class="category-card__primary-image">
              <a href="/fast-food/mcdonalds-check-employees-health-work">
               <img alt="The exterior of a McDonald's restaurant." src="https://www.qsrmagazine.com/sites/default/files/styles/media_thumbnail/public/story/mcdonalds-becomes-100-billion-company.jpg?itok=Tgew6Ytx"/>
              </a>
             </div>
             <div class="category-card__primary-title">
              <a href="/fast-food/mcdonalds-check-employees-health-work" hreflang="en">
               McDonald’s to Check Employees’ Health Before Work
              </a>
             </div>
             <div class="category-card__other-title category-card__other-title-1">
              <a href="/legal/restaurants-theres-big-problem-stimulus-package" hreflang="en">
               For Restaurants, there’s a Big Problem with the Stimulus Package
              </a>
         

In [6]:
chart = soup.find(id="block-qsr2020-content")
data_company = [dc.get_text() for dc in chart.select(".sortable-chart-container .data-chain")]
data_category = [dct.get_text() for dct in chart.select(".sortable-chart-container .data-segment")]
all_sales = [als.get_text() for als in chart.select(".sortable-chart-container .data-2012_sales")]
avg_sales = [avs.get_text() for avs in chart.select(".sortable-chart-container .data-2012_avg_sales")]
franchise_units = [fu.get_text() for fu in chart.select(".sortable-chart-container .data-2012_franchise_units")]
company_units = [cu.get_text() for cu in chart.select(".sortable-chart-container .data-2012_company_units")]
total_units = [tu.get_text() for tu in chart.select(".sortable-chart-container .data-2012_total_units")]

In [7]:
import pandas as pd
restaurants2012 = pd.DataFrame({
    "Company": data_company,
    "Category": data_category,
    "Systemwide_Sales_Millions": all_sales,
    "Avg_Sales_per_Unit_Thousands": avg_sales,
    "Franchise_Units": franchise_units,
    "Company_Units": company_units,
    "Total_Units": total_units
})

In [8]:
restaurants2012.head()

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units
0,McDonald's,burger,"$35,600.0","$2,600.0","12,605","1,552","14,157"
1,Subway*,sandwich,"$12,100.0",$481.0,"25,549",0,"25,549"
2,Starbucks*<sup>1</sup>,snack,"$10,600.0","$1,223.0","4,262","6,866","11,128"
3,Wendy's<sup>2</sup>,burger,"$8,600.0","$1,483.8","4,528","1,289","5,817"
4,Burger King*,burger,"$8,587.0","$1,195.0","7,000",183,"7,183"


In [9]:
# transform data by setting the index
transformed_restaurants2012 = restaurants2012.copy()

In [10]:
transformed_restaurants2012["Company"].is_unique

True

In [11]:
transformed_restaurants2012.head()

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units
0,McDonald's,burger,"$35,600.0","$2,600.0","12,605","1,552","14,157"
1,Subway*,sandwich,"$12,100.0",$481.0,"25,549",0,"25,549"
2,Starbucks*<sup>1</sup>,snack,"$10,600.0","$1,223.0","4,262","6,866","11,128"
3,Wendy's<sup>2</sup>,burger,"$8,600.0","$1,483.8","4,528","1,289","5,817"
4,Burger King*,burger,"$8,587.0","$1,195.0","7,000",183,"7,183"


In [12]:
transformed_restaurants2012 = transformed_restaurants2012.assign(id=(transformed_restaurants2012['Company']).astype('category').cat.codes)
# transformed_restaurants    

In [13]:
transformed_restaurants2012.get_dtype_counts()

object    7
int8      1
dtype: int64

In [14]:
transformed_restaurants2012.loc["McDonald's" : , "Systemwide_Sales_Millions"]


Series([], Name: Systemwide_Sales_Millions, dtype: object)

In [15]:
extr = transformed_restaurants2012["Systemwide_Sales_Millions"]
type(extr) is tuple

False

In [16]:
transformed_restaurants2012["Company"] = transformed_restaurants2012["Company"].str.strip()
transformed_restaurants2012["Category"] = transformed_restaurants2012["Category"].str.strip()
transformed_restaurants2012["Systemwide_Sales_Millions"] = transformed_restaurants2012["Systemwide_Sales_Millions"].str.strip()
transformed_restaurants2012["Avg_Sales_per_Unit_Thousands"] = transformed_restaurants2012["Avg_Sales_per_Unit_Thousands"].str.strip()
transformed_restaurants2012["Franchise_Units"] = transformed_restaurants2012["Franchise_Units"].str.strip()
transformed_restaurants2012["Company_Units"] = transformed_restaurants2012["Company_Units"].str.strip()
transformed_restaurants2012["Total_Units"] = transformed_restaurants2012["Total_Units"].str.strip()
transformed_restaurants2012.head()

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units,id
0,McDonald's,burger,"$35,600.0","$2,600.0","12,605","1,552","14,157",29
1,Subway*,sandwich,"$12,100.0",$481.0,"25,549",0,"25,549",42
2,Starbucks*<sup>1</sup>,snack,"$10,600.0","$1,223.0","4,262","6,866","11,128",40
3,Wendy's<sup>2</sup>,burger,"$8,600.0","$1,483.8","4,528","1,289","5,817",45
4,Burger King*,burger,"$8,587.0","$1,195.0","7,000",183,"7,183",4


In [17]:
transformed_restaurants2012["Systemwide_Sales_Millions"] = transformed_restaurants2012["Systemwide_Sales_Millions"].str.replace('$','',regex=False)
transformed_restaurants2012["Systemwide_Sales_Millions"] = transformed_restaurants2012["Systemwide_Sales_Millions"].str.replace(',','',regex=False).astype(float)
transformed_restaurants2012["Avg_Sales_per_Unit_Thousands"] = transformed_restaurants2012["Avg_Sales_per_Unit_Thousands"].str.replace('$','',regex=False)
transformed_restaurants2012["Avg_Sales_per_Unit_Thousands"] = transformed_restaurants2012["Avg_Sales_per_Unit_Thousands"].str.replace(',','',regex=False).astype(float)
transformed_restaurants2012["Franchise_Units"] = transformed_restaurants2012["Franchise_Units"].str.replace(',','',regex=False).astype(float)
transformed_restaurants2012["Company_Units"] = transformed_restaurants2012["Company_Units"].str.replace(',','',regex=False).astype(float)
transformed_restaurants2012["Total_Units"] = transformed_restaurants2012["Total_Units"].str.replace(',','',regex=False).astype(float)
transformed_restaurants2012["Company"] = transformed_restaurants2012["Company"].str.replace('<sup>1</sup>','',regex=False)
transformed_restaurants2012["Company"] = transformed_restaurants2012["Company"].str.replace('<sup>2</sup>','',regex=False)
transformed_restaurants2012["Company"] = transformed_restaurants2012["Company"].str.replace('*','',regex=False)

In [18]:
transformed_restaurants2012.head()

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units,id
0,McDonald's,burger,35600.0,2600.0,12605.0,1552.0,14157.0,29
1,Subway,sandwich,12100.0,481.0,25549.0,0.0,25549.0,42
2,Starbucks,snack,10600.0,1223.0,4262.0,6866.0,11128.0,40
3,Wendy's,burger,8600.0,1483.8,4528.0,1289.0,5817.0,45
4,Burger King,burger,8587.0,1195.0,7000.0,183.0,7183.0,4


In [19]:
import sqlite3

In [20]:
conn = sqlite3.connect('restaurants2012.sqlite')
c = conn.cursor()

In [21]:
c.execute('CREATE TABLE rest_table (Company text, Category text, Systemwide_Sales_Millions number, Avg_Sales_per_Unit_Thousands number, Franchise_Units number, Company_Units number, Total_Units number, [id] number PRIMARY KEY)')
conn.commit()

In [22]:
transformed_restaurants2012.to_sql('rest_table', conn, if_exists='replace', index=False)

In [23]:
c.execute('''  
SELECT * FROM rest_table
          ''')

In [24]:
for row in c.fetchall():
    print (row)

("McDonald's", 'burger', 35600.0, 2600.0, 12605.0, 1552.0, 14157.0, 29)
('Subway', 'sandwich', 12100.0, 481.0, 25549.0, 0.0, 25549.0, 42)
('Starbucks', 'snack', 10600.0, 1223.0, 4262.0, 6866.0, 11128.0, 40)
("Wendy's", 'burger', 8600.0, 1483.8, 4528.0, 1289.0, 5817.0, 45)
('Burger King', 'burger', 8587.0, 1195.0, 7000.0, 183.0, 7183.0, 4)
('Taco Bell', 'mexican', 7478.0, 1363.0, 4218.0, 1044.0, 5262.0, 43)
("Dunkin' Donuts", 'snack', 6264.2, 857.4, 7278.0, 28.0, 7306.0, 16)
('Pizza Hut', 'pizza/pasta', 5666.0, 883.0, 5757.0, 452.0, 6209.0, 35)
('Chick-fil-A', 'chicken', 4621.1, 3157.9, 1391.0, 292.0, 1683.0, 8)
('KFC', 'chicken', 4459.0, 957.0, 4319.0, 237.0, 4556.0, 25)
('Panera Bread', 'sandwich', 3861.0, 2427.2, 843.0, 809.0, 1652.0, 32)
('Sonic Drive-In', 'burger', 3790.7, 1074.0, 3147.0, 409.0, 3556.0, 39)
("Domino's Pizza", 'pizza/pasta', 3500.0, 710.2, 4540.0, 388.0, 4928.0, 15)
('Jack in the Box', 'burger', 3084.9, 1379.0, 1703.0, 547.0, 2250.0, 21)
("Arby's", 'sandwich', 2992.

In [25]:
transformed_restaurants2012.head()

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units,id
0,McDonald's,burger,35600.0,2600.0,12605.0,1552.0,14157.0,29
1,Subway,sandwich,12100.0,481.0,25549.0,0.0,25549.0,42
2,Starbucks,snack,10600.0,1223.0,4262.0,6866.0,11128.0,40
3,Wendy's,burger,8600.0,1483.8,4528.0,1289.0,5817.0,45
4,Burger King,burger,8587.0,1195.0,7000.0,183.0,7183.0,4


In [26]:
# Creating dataframe that shows only Burger Category 
burgers = transformed_restaurants2012.loc[transformed_restaurants2012["Category"]=="burger",]
top_burgers = burgers.iloc[:3]
top_burgers

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units,id
0,McDonald's,burger,35600.0,2600.0,12605.0,1552.0,14157.0,29
3,Wendy's,burger,8600.0,1483.8,4528.0,1289.0,5817.0,45
4,Burger King,burger,8587.0,1195.0,7000.0,183.0,7183.0,4


In [27]:
# Creating dataframe that shows only Chicken Category 
chicken = transformed_restaurants2012.loc[transformed_restaurants2012["Category"]=="chicken",]
top_chicken = chicken.iloc[:3]
top_chicken

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units,id
8,Chick-fil-A,chicken,4621.1,3157.9,1391.0,292.0,1683.0,8
9,KFC,chicken,4459.0,957.0,4319.0,237.0,4556.0,25
18,Popeyes Louisiana Kitchen,chicken,2253.0,1242.0,1634.0,45.0,1679.0,36


In [28]:
# Creating dataframe that shows only Sandwich Category 
Sandwich = transformed_restaurants2012.loc[transformed_restaurants2012["Category"]=="sandwich",]
top_Sandwich = Sandwich.iloc[:3]
top_Sandwich

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units,id
1,Subway,sandwich,12100.0,481.0,25549.0,0.0,25549.0,42
10,Panera Bread,sandwich,3861.0,2427.2,843.0,809.0,1652.0,32
14,Arby's,sandwich,2992.0,993.2,2343.0,1011.0,3354.0,0


In [29]:
# Creating dataframe that shows only Pizza Category 
Pizza = transformed_restaurants2012.loc[transformed_restaurants2012["Category"]=="pizza/pasta",]
top_Pizza = Pizza.iloc[:3]
top_Pizza

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units,id
7,Pizza Hut,pizza/pasta,5666.0,883.0,5757.0,452.0,6209.0,35
12,Domino's Pizza,pizza/pasta,3500.0,710.2,4540.0,388.0,4928.0,15
16,Papa John's,pizza/pasta,2402.4,829.0,2483.0,648.0,3131.0,33


In [30]:
# Creating dataframe that shows only Snack Category 
Snack = transformed_restaurants2012.loc[transformed_restaurants2012["Category"]=="snack",]
top_Snack = Snack.iloc[:3]
top_Snack

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units,id
2,Starbucks,snack,10600.0,1223.0,4262.0,6866.0,11128.0,40
6,Dunkin' Donuts,snack,6264.2,857.4,7278.0,28.0,7306.0,16
39,Krispy Kreme,snack,569.4,2428.0,142.0,97.0,239.0,26


In [31]:
total_by_category = transformed_restaurants2012.groupby(["Category"])[["Systemwide_Sales_Millions"]].sum()
total_by_category = total_by_category.reset_index()
total_by_category

,Category,Systemwide_Sales_Millions
0,asian,1797.4
1,burger,71361.8
2,chicken,15619.7
3,mexican,11852.3
4,pizza/pasta,14495.4
5,sandwich,21632.7
6,seafood,1190.1
7,snack,18925.8


In [32]:
merged_top_df = pd.concat([top_burgers, top_chicken, top_Sandwich, top_Pizza, top_Snack])
merged_top_df

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units,id
0,McDonald's,burger,35600.0,2600.0,12605.0,1552.0,14157.0,29
3,Wendy's,burger,8600.0,1483.8,4528.0,1289.0,5817.0,45
4,Burger King,burger,8587.0,1195.0,7000.0,183.0,7183.0,4
8,Chick-fil-A,chicken,4621.1,3157.9,1391.0,292.0,1683.0,8
9,KFC,chicken,4459.0,957.0,4319.0,237.0,4556.0,25
18,Popeyes Louisiana Kitchen,chicken,2253.0,1242.0,1634.0,45.0,1679.0,36
1,Subway,sandwich,12100.0,481.0,25549.0,0.0,25549.0,42
10,Panera Bread,sandwich,3861.0,2427.2,843.0,809.0,1652.0,32
14,Arby's,sandwich,2992.0,993.2,2343.0,1011.0,3354.0,0
7,Pizza Hut,pizza/pasta,5666.0,883.0,5757.0,452.0,6209.0,35


In [33]:
# Selecting the top company for each category
top_rest_per_category = merged_top_df.loc[(merged_top_df["Company"] == "McDonald's") | (merged_top_df["Company"] == "Chick-fil-A") | (merged_top_df["Company"] == "Subway*") | (merged_top_df["Company"] == "Pizza Hut") | (merged_top_df["Company"] == "Taco Bell") | (merged_top_df["Company"] == "Starbucks*<sup>1</sup>")]
top_rest_per_category

,Company,Category,Systemwide_Sales_Millions,Avg_Sales_per_Unit_Thousands,Franchise_Units,Company_Units,Total_Units,id
0,McDonald's,burger,35600.0,2600.0,12605.0,1552.0,14157.0,29
8,Chick-fil-A,chicken,4621.1,3157.9,1391.0,292.0,1683.0,8
7,Pizza Hut,pizza/pasta,5666.0,883.0,5757.0,452.0,6209.0,35


In [34]:
c.execute('''CREATE TABLE top_rest ([Company] text, [Category] text, [Systemwide_Sales_Millions] number, [Avg_Sales_per_Unit_Thousands] number, [Franchise_Units] number, [Company_Units] number, [Total_Units] number, [id] INTEGER PRIMARY KEY)''')
conn.commit()

In [35]:
merged_top_df.to_sql('top_rest', conn, if_exists='replace', index=False)

In [36]:
c.execute('''  
SELECT * FROM top_rest
          ''')

In [37]:
for row in c.fetchall():
    print (row)

("McDonald's", 'burger', 35600.0, 2600.0, 12605.0, 1552.0, 14157.0, 29)
("Wendy's", 'burger', 8600.0, 1483.8, 4528.0, 1289.0, 5817.0, 45)
('Burger King', 'burger', 8587.0, 1195.0, 7000.0, 183.0, 7183.0, 4)
('Chick-fil-A', 'chicken', 4621.1, 3157.9, 1391.0, 292.0, 1683.0, 8)
('KFC', 'chicken', 4459.0, 957.0, 4319.0, 237.0, 4556.0, 25)
('Popeyes Louisiana Kitchen', 'chicken', 2253.0, 1242.0, 1634.0, 45.0, 1679.0, 36)
('Subway', 'sandwich', 12100.0, 481.0, 25549.0, 0.0, 25549.0, 42)
('Panera Bread', 'sandwich', 3861.0, 2427.2, 843.0, 809.0, 1652.0, 32)
("Arby's", 'sandwich', 2992.0, 993.2, 2343.0, 1011.0, 3354.0, 0)
('Pizza Hut', 'pizza/pasta', 5666.0, 883.0, 5757.0, 452.0, 6209.0, 35)
("Domino's Pizza", 'pizza/pasta', 3500.0, 710.2, 4540.0, 388.0, 4928.0, 15)
("Papa John's", 'pizza/pasta', 2402.4, 829.0, 2483.0, 648.0, 3131.0, 33)
('Starbucks', 'snack', 10600.0, 1223.0, 4262.0, 6866.0, 11128.0, 40)
("Dunkin' Donuts", 'snack', 6264.2, 857.4, 7278.0, 28.0, 7306.0, 16)
('Krispy Kreme', 'sna

In [38]:
c.execute('CREATE TABLE total_by_category (Company text, Category text, Systemwide_Sales_Millions number, Avg_Sales_per_Unit_Thousands number, Franchise_Units number, Company_Units number, Total_Units number, [id] number PRIMARY KEY)')
conn.commit()

In [39]:
total_by_category.to_sql('total_by_category', conn, if_exists='replace', index=False)

In [40]:
c.execute('''  
SELECT * FROM total_by_category
          ''')

In [41]:
for row in c.fetchall():
    print (row)

('asian', 1797.4)
('burger', 71361.79999999999)
('chicken', 15619.699999999999)
('mexican', 11852.300000000001)
('pizza/pasta', 14495.4)
('sandwich', 21632.7)
('seafood', 1190.1)
('snack', 18925.800000000003)


In [42]:
c.execute('CREATE TABLE top_rest_per_category (Company text, Category text, Systemwide_Sales_Millions number, Avg_Sales_per_Unit_Thousands number, Franchise_Units number, Company_Units number, Total_Units number, [id] number PRIMARY KEY)')
conn.commit()

In [43]:
top_rest_per_category.to_sql('top_rest_per_category', conn, if_exists='replace', index=False)

In [44]:
c.execute('''  
SELECT * FROM top_rest_per_category
          ''')

In [45]:
for row in c.fetchall():
    print (row)

("McDonald's", 'burger', 35600.0, 2600.0, 12605.0, 1552.0, 14157.0, 29)
('Chick-fil-A', 'chicken', 4621.1, 3157.9, 1391.0, 292.0, 1683.0, 8)
('Pizza Hut', 'pizza/pasta', 5666.0, 883.0, 5757.0, 452.0, 6209.0, 35)
